In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
from scipy.stats import (
    shapiro, kstest, levene, bartlett, kruskal,
    mannwhitneyu, f_oneway, ranksums)
from statsmodels.stats.multitest import multipletests
import statsmodels.api
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import scanpy as sc
import scanpy.external as sce
import anndata as ad
import gseapy as gp
from gseapy import barplot, dotplot
from gseapy.parser import read_gmt
import itertools
import random
import harmonypy as hm

#### Data Cleaning

In [ ]:
IDmap = pd.read_csv(r'C:\Users\momenzadeha\OneDrive - Cedars-Sinai Health System\jian\idmapping_human.tsv', sep="\t")
IDmap.rename(columns={'From': 'protein'},inplace=True)

In [ ]:
def make_diann_matrix():
    df = pd.read_parquet('report_diann_pusc.parquet')
    df = df[df['Global.PG.Q.Value'] < 0.01]
    df.loc[:, 'PG.Quantity'] = df['PG.MaxLFQ'].values
    df = df[['Protein.Group', 'Run', 'PG.Quantity']]
    df = df.drop_duplicates().reset_index(drop=True)
    df = df[~df['Protein.Group'].str.contains(';')] # groups not in considering?
    df = df.pivot(index='Protein.Group', columns='Run', values='PG.Quantity')
    df = df.reset_index()
    return df
diann_df=make_diann_matrix()
diann_df=diann_df.set_index('Protein.Group')
diann_df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
# Build metadata from original column names
sample_metadata = pd.DataFrame({
    'Sample': diann_df.columns,
    'Group': [col.split('_')[1] if len(col.split('_')) > 1 else None for col in diann_df.columns],
    'Batch': [col.split('_')[3] if len(col.split('_')) > 3 else None for col in diann_df.columns]
})

# Transpose protein_beta so that each row becomes a sample
protein_long = diann_df.T.copy()  # Transpose
protein_long.index.name = 'Sample'   # Set index name so it can be used as a column
protein_long = protein_long.reset_index()  # Make index into column

# Merge the metadata
protein_annotated = protein_long.merge(sample_metadata, on='Sample', how='left')
protein_annotated["Group"] = protein_annotated["Group"].replace({
    "fix": "Fixed",
    "migrate": "Migratory"})

In [ ]:
# Step 1: Separate metadata
meta_cols = ['Sample', 'Batch', 'Group']
metadata = protein_annotated[meta_cols].copy()
expr_df = protein_annotated.drop(['Batch','Group'], axis=1).copy()
expr_df=expr_df.set_index('Sample')
metadata=metadata.set_index('Sample')
adata = sc.AnnData(expr_df.copy())
adata.obs = metadata.loc[adata.obs_names].copy()

# Step 2: Filter proteins with >75% missing values
protein_missing = np.isnan(adata.X).mean(axis=0)
adata = adata[:, protein_missing <= 0.75].copy()

# Step 3: Filter proteins quantified in <3 cells
valid_proteins = (np.sum(~np.isnan(adata.X), axis=0) >= 3)
adata = adata[:, valid_proteins].copy()

# Step 4: Fill missing values with zeros
adata.X = np.nan_to_num(adata.X, nan=0.0)

# Step 5: Normalize protein quantities to sum to 10,000 per cell
sc.pp.normalize_total(adata, target_sum=1e4)

# Step 6: Natural log transform
sc.pp.log1p(adata)

# Step 7: PCA
sc.tl.pca(adata)

# Step 8: Run Harmony using the PCA coordinates
ho = hm.run_harmony(adata.obsm['X_pca'], adata.obs, 'Batch')

# Replace PCA coordinates with Harmony-corrected ones
adata.obsm['X_pca_harmony'] = ho.Z_corr.T  # transpose to match AnnData shape

sc.pp.neighbors(adata, use_rep='X_pca_harmony')
sc.tl.umap(adata,n_components=25, min_dist=0.1, spread=1.0)

# Step 9: Louvain clustering
sc.tl.leiden(adata, resolution=0.7)

In [ ]:
# Create a crosstab of Group vs Leiden cluster
group_counts = pd.crosstab(adata.obs['leiden'], adata.obs['Group'])

# Calculate percent within each Leiden cluster (row-wise)
group_percent = group_counts.div(group_counts.sum(axis=1), axis=0) * 100

# Optional: round for readability
group_percent = group_percent.round(2)

# Display
print(group_percent)

In [ ]:
# Differential protein analysis - compare Leiden clusters 0 and 2 (purest fixed vs migratory)

adata_avb = adata[adata.obs['leiden'].isin(['0', '2'])].copy()
X = adata_avb.X.toarray() if hasattr(adata_avb.X, 'toarray') else adata_avb.X
X_df = pd.DataFrame(X, columns=adata_avb.var_names, index=adata_avb.obs_names)
group_labels = adata_avb.obs['leiden']
groupa_idx = group_labels[group_labels == '0'].index
groupb_idx = group_labels[group_labels == '2'].index
genes = []
lnfc = []
pvals = []

# Loop through features (genes/proteins)
for gene in X_df.columns:
    xa = X_df.loc[groupa_idx, gene]
    xb = X_df.loc[groupb_idx, gene]
    
    # Filter: keep genes with ≥3 non-zero values in both groups
    if (xa > 0).sum() < 2 or (xb > 0).sum() < 2:
        continue
    
    # Log10 Fold Change = mean(log10 expr in 0) - mean(log10 expr in 2)
    lnfc.append(xa.mean() - xb.mean())

    # Wilcoxon test
    stat, p = ranksums(xa, xb)
    pvals.append(p)
    genes.append(gene)

# Adjust p-values using Benjamini-Hochberg
pvals_adj = multipletests(pvals, method='fdr_bh')[1]

# Compile results
volcano_df = pd.DataFrame({
    'gene': genes,
    'lnFC': lnfc,
    'pval_adj': pvals_adj,})
volcano_df['neg_log10_pval'] = -np.log10(volcano_df['pval_adj'].replace(0, np.nan))

# Volcano plot
plt.figure(figsize=(5, 4))
sns.scatterplot(
    data=volcano_df,
    x='lnFC',
    y='neg_log10_pval',
    hue=volcano_df['pval_adj'] < 0.05,
    palette={True: 'crimson', False: 'gray'},
    legend=False
)

plt.axhline(1.3, ls='--', color='black')
plt.title('Cluster 0 vs 2',fontsize=30)
plt.xlabel('Ln(FC)',fontsize=22)
plt.ylabel(r"$-\log_{10}(\mathrm{adj.\ p\text{-}value})$", fontsize=20)
plt.xlim(-1.5,1.5)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.tight_layout()
plt.show()

In [ ]:
# Count DEPs based on thresholds
dep_mask = volcano_df['pval_adj'] < 0.05
n_deps = dep_mask.sum()
print(f"Number of differentially expressed proteins (DEPs): {n_deps}")

In [ ]:
leidavb=volcano_df[(volcano_df['pval_adj'] < 0.05)]

In [ ]:
IDmap = IDmap.rename(columns={'protein': 'gene'})
leidavb=leidavb.merge(IDmap, on='gene',how='inner')

In [ ]:
# Prepare  ranked list and save to ranked list for GSEApy (descending by LogFC)
rnk_df = (
    leidavb
      .loc[:, ['Gene Names (primary)', 'lnFC']]
      .rename(columns={'Gene Names (primary)': 'Term',
                       'lnFC':'Score'}))

ranked_genes = rnk_df[['Term', 'Score']].sort_values('Score', ascending=False)
ranked_genes.to_csv("ranked_genes.rnk", sep="\t", index=False, header=False)

#### GPT-o4-mini-high

In [ ]:
gene_sets = read_gmt("KEGG_2021_Human.gmt")

In [ ]:
random.seed(42)
np.random.seed(42)
genes = ranked_genes["Term"].tolist()
scores = ranked_genes["Score"].tolist()
score_dict = {g: abs(s) for g, s in zip(genes, scores)}

# 1. Enrichment Score (weighted)
def enrichment_score(gene_list, gene_set):
    hits = np.array([1 if g in gene_set else 0 for g in gene_list])
    Nh = hits.sum()
    Nm = len(gene_list) - Nh
    if Nh < 15 or Nh > 500 or Nh == 0:
        return 0.0
    hit_w = np.array([score_dict[g] for g in gene_list]) * hits
    sum_hit_w = hit_w.sum()
    hit_scores = hit_w / sum_hit_w
    miss_scores = (1 - hits) / Nm
    running = np.cumsum(hit_scores - miss_scores)
    max_es, min_es = running.max(), running.min()
    return max_es if abs(max_es) >= abs(min_es) else min_es

# 2. Null distribution & NES/p-value with pseudocount
def null_distribution_pseudo(gene_list, gene_set, n_perm=20):
    hits_orig = [1 if g in gene_set else 0 for g in gene_list]
    null_es = []
    for _ in range(n_perm):
        perm_hits = random.sample(hits_orig, k=len(hits_orig))
        hit_w = np.array([score_dict[g] for g in gene_list]) * perm_hits
        sum_hit_w = hit_w.sum() or 1.0
        hit_scores = hit_w / sum_hit_w
        miss_scores = [(1 - h) / (len(gene_list) - sum(perm_hits)) for h in perm_hits]
        rs = []
        running = 0.0
        for hs, ms in zip(hit_scores, miss_scores):
            running += hs - ms
            rs.append(running)
        max_es, min_es = max(rs), min(rs)
        null_es.append(max_es if abs(max_es) >= abs(min_es) else min_es)
    return np.array(null_es)

# 3. Compute ES, NES, p-value for each pathway
results = []
n_perm = 1000

for pathway, members in gene_sets.items():
    ES_obs = enrichment_score(genes, members)
    if ES_obs == 0.0:
        continue
    null_es = null_distribution_pseudo(genes, members, n_perm=n_perm)
    if ES_obs >= 0:
        p_val = (np.sum(null_es >= ES_obs) + 1) / (n_perm + 1)
        mean_pos = null_es[null_es >= 0].mean() or 1.0
        NES = ES_obs / mean_pos
    else:
        p_val = (np.sum(null_es <= ES_obs) + 1) / (n_perm + 1)
        mean_neg = abs(null_es[null_es < 0].mean()) or 1.0
        NES = ES_obs / mean_neg

    results.append((pathway, ES_obs, NES, p_val))

df = pd.DataFrame(results, columns=["Pathway", "ES", "NES", "p_val"])

# 4. FDR q-value (Benjamini-Hochberg)
df = df.sort_values("p_val").reset_index(drop=True)
m = len(df)
df["rank"] = np.arange(1, m + 1)
df["q_val"] = (df["p_val"] * m / df["rank"]).clip(upper=1.0)
for i in range(m - 2, -1, -1):
    df.at[i, "q_val"] = min(df.at[i, "q_val"], df.at[i + 1, "q_val"])

# 5. Select top 15 and plot
top15_gpt = df.nsmallest(15, "q_val").copy()
top15_gpt = top15_gpt.sort_values("q_val")

plt.figure(figsize=(8, 6))
plt.barh(top15_gpt["Pathway"], top15_gpt["NES"])
plt.xlabel("NES", fontsize=22)
plt.title("GPT-o4-mini-high", fontsize=22)

# Increase tick label font sizes
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.tight_layout()
plt.show()

#### preranked GSEA package

In [ ]:
# 1. Run preranked GSEA
pre_res = gp.prerank(
    rnk=ranked_genes,       
    gene_sets=gene_sets,  
    outdir=None,                     
    permutation_num=1000,         
    min_size=15,                      
    max_size=500,                     
    seed=42,                           
    verbose=True
)

# 2. Extract and format results
res = pre_res.res2d.copy()
res["q_val"] = pd.to_numeric(res["FDR q-val"], errors="raise")

# 3. Compute –log10(FDR q-value)
eps = np.nextafter(0, 1)
res["-log10(q_val)"] = -np.log10(res["q_val"].replace(0, eps))

# 7. Select top 15 and plot
top15_pkg = res.nsmallest(15, "q_val").copy()
top15_pkg = top15_pkg.sort_values("q_val")

plt.figure(figsize=(8, 6))
plt.barh(top15_pkg["Term"], top15_pkg["NES"])
plt.xlabel("NES", fontsize=22)
plt.title("GSEAPy", fontsize=22)

# Increase tick label font sizes
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.tight_layout()
plt.show()

In [ ]:
# Merge on pathway name
merged = pd.merge(top15_gpt, top15_pkg, left_on="Pathway", right_on="Term", suffixes=("_manual", "_gseapy"))

# Scatterplot
plt.figure(figsize=(6, 6))
plt.scatter(merged["q_val_manual"], merged["q_val_gseapy"], color='blue')

plt.xlabel("GPT-o4-mini-high q value", fontsize=22)
plt.ylabel("GSEAPy q value", fontsize=22)

# Increase tick label font sizes
plt.xticks(fontsize=18, rotation=45)
plt.yticks(fontsize=20)
plt.tight_layout()
plt.show()